In [1]:
#!pip install pymysql

In [2]:
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb

import pandas as pd
from sqlalchemy import create_engine

from config import db_username, db_password, db_host, db_port, db_name

ModuleNotFoundError: No module named 'config'

### Store CSV into DataFrame

In [ ]:
csv_file = "../Resources/customer_data.csv"
customer_data_df = pd.read_csv(csv_file)
customer_data_df.head()

### Create new data with select columns

In [ ]:
new_customer_data_df = customer_data_df[['customer_id', 'first_name', 'last_name']]
new_customer_data_df.head()

### Store JSON data into a DataFrame

In [ ]:
json_file = "../Resources/customer_location.json"
customer_location_df = pd.read_json(json_file)
customer_location_df.head()

### Clean DataFrame

In [ ]:
new_customer_location_df = customer_location_df[["id", "address", "us_state"]]
new_customer_location_df.rename(columns={'id':'customer_id'}, inplace=True)
new_customer_location_df.head()

### Connect to Cloud Database

`dialect`://`username`:`password`@`hostname`:`port`/`databasename`

In [ ]:
rds_connection_string = f'{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(f'mysql://{rds_connection_string}')
conn = engine.connect()

print(f'mysql://{rds_connection_string}')

### Check for tables

In [ ]:
query = '''
    SELECT
        TABLE_SCHEMA,
        TABLE_NAME,
        CREATE_TIME
    FROM
        information_schema.tables
    WHERE
        TABLE_TYPE = 'BASE TABLE'
        AND TABLE_SCHEMA = 'customer_db';
'''

tables_df = pd.read_sql(query, conn)

tables_df.head(20)

### Use pandas to load csv converted DataFrame into database

In [ ]:
new_customer_data_df['dartling_name'] = 'Kwame 6'
new_customer_data_df.head()

In [ ]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [ ]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using MySQL workbench

In [ ]:
data_df = pd.read_sql('SELECT * from customer_name', con=conn)
data_df.head()

In [ ]:
data_df['dartling_name'].unique()

### Confirm data has been added by querying the customer_location table

In [ ]:
pd.read_sql('select * from customer_location', con=engine).head()